# RNN (순환 신경망)
- 이전의 시점의 정보를 현재 시점에 계산에 반영하는 구조

- 첫번째 입력이 들어왔을때 
    - 초기의 상태 (h_0 = 0)와 입력에 대한 상태 (h_1)로 계산
- 두번째 입력이 들어왔을때
    - 이전의 상태 (h_1)과 결합해서 계산 (h_2)
- 반복 작업이 완료가 되었을때
    - 마지막 상태 (h_T) 생성 

- 매개변수 
    - input_size 
        - 각 시점에서 입력의 피쳐의 개수 
    - hidden_size
        - 은닉층(출력)의 크기 
    - num_layers
        - 기본값 : 1
        - RNN 층의 개수 (층이 깊을수록 복잡한 패턴이 생성)
        - 개수가 늘어날수록 시간이 증가하고 과적합의 위험성이 존재
        - 1~3 정도 사용을 할때 1부터 개수를 1씩 늘려가면서 사용
    - nonlinearity
        - 기본값 : 'tanh'
        - 비선형 활성화 함수를 선택 
    - bias
        - 기본값 : True
        - 각 가중치에 편향 항을 추가 여부 
    - batch_first
        - 기본값 : False
        - 입력 텐서 첫 차원이 배치인지 여부 
    - dropout
        - 기본값 : 0.0 
        - 층 사이에 드랍아웃의 적용 비율 -> 층이 2개 이상에 사용
    - bidirectional
        - 기본값 : False
        - 양방향 RNN을 사용할지 여부 (순방향 이 기본 , 역방향을 사용할지 지정)

- 매개변수의 유의점
    - hidden_size
        - 너무 적은 경우라면 정보가 부족, 너무 큰 경우에는 과적합 
        - 일반적으로는 32 ~ 128
    - num_layers
        - 시간의 복잡도에 따라 1 ~ 3정도 사용
        - 1부터 테스트를 돌려서 교차 검증
    - nolinearity
        - 'tanh'이 기본 값이 'relu'로 변경하게 되면 시간은 감소할 수 있지만 불안정 
    - dropout
        - 층이 2개인 경우에 사용
        - 과적합 방지를 위해 0.2 ~ 0.5 사용 (권장)

In [2]:
# RNN 기본형 
# 샘플 데이터를 생성해서 코드 작성 
import math
import numpy as np 
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [3]:
# 랜덤 일관성 설정 
np.random.seed(42)
torch.manual_seed(42)

In [5]:
# 사인 그래프를 생성 + 노이즈 

idx = torch.arange(3000).float()
data = torch.sin(2 * math.pi * 0.02 * idx) + 0.05 * torch.rand(3000)

In [6]:
data

tensor([ 0.0441,  0.1711,  0.2678,  ..., -0.3259, -0.2480, -0.0964])

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(idx, data)
plt.show()

In [13]:
# 연속성이 있는 시계열데이터를 기준으로 train, test를 8:2의 비율로 나눠준다. 
# 앞부분 80% , 뒷부분 20%
# 전체 데이터의 길이에서 0.8을 곱한 idx의 값이 데이터의 경계
split_idx = int(len(data) * 0.8)
train_data = data[ : split_idx]         # 학습 데이터(구간)
test_data = data[split_idx : ]          # 평가 데이터(구간) 


In [15]:
# train와 test를 스케일링 
scaler = StandardScaler()

train_data = scaler.fit_transform(train_data.reshape(-1, 1))
test_data = scaler.transform(test_data.reshape(-1, 1))

In [ ]:
test_data

In [17]:
# data들을 torch에서 사용하기 위한 tensor의 형태로 변환 (2차원의 데이터를 1차원의 데이터로 변환)
train_data = torch.tensor(train_data.squeeze(-1), dtype=torch.float32)
test_data = torch.tensor(test_data.squeeze(-1), dtype=torch.float32)

In [18]:
class WindowDataset(Dataset):
    # 단변량 시계열에서 입력 값 정답 값을 만드는 Dataset
    def __init__(self, _data, _window):
        # _data : (N, ) 형태의 1차원 tensor 데이터
        # _window : 과거의 몇개의 데이터를 볼것인가?(구간 설정)
        self.data = _data
        self.window = _window
        # 유효 샘플의 개수는 학습 데이터의 개수는 data의 전체 길이에서 -1
        # 입력 데이터는 전체 길이 - 윈도우 
        self.n = len(_data) - _window

    # __len__(), __getitem__() 두개의 특수 함수들은 
    # DataLoder에서 자동으로 호출해서 사용이 되는 부분
    def __len__(self):
        return max(self.n, 0)
    
    def __getitem__(self, idx):
        # 변환
        # x -> 입력 데이터 (윈도우의 구간을 나타내는 데이터) --> 문제
        # y -> 입력 데이터 다음 행의 데이터  --> 정답
        x = self.data[idx : idx + self.window].unsqueeze(-1) # (window, ) -> (window, 1)
        y = self.data[idx + self.window].unsqueeze(-1)
        return x, y

In [19]:
# class 생성
# 구간 설정 값 
window = 50
train_ds = WindowDataset(train_data, window)
val_ds = WindowDataset(test_data, window)